<a href="https://colab.research.google.com/github/bhosalemahesh899/AIPractice/blob/main/FetchCategoriesAndSubCategories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#!pip install --upgrade openai httpx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [1]:
#!pip install --upgrade openai httpx==0.27.2 # Downgrade httpx to a compatible version
!pip install openai httpx==0.27.2 # --force-reinstall

from openai import OpenAI
from google.colab import userdata
key = userdata.get('cysOpenAIClass')
client = OpenAI(api_key=key)

assistant = client.beta.assistants.create(
name="Financial Analyst Assistant",
instructions="You are an expert open answer survay analyst. Use you knowledge base to answer questions about the categories and subcategories and keywords.",
model="gpt-4o",
tools=[{"type": "file_search"}],
)

In [2]:
# Create a vector store caled "Open answers or comments"
vector_store = client.beta.vector_stores.create(name="Open answers or comments")

# Ready the files for upload to OpenAI
file_paths = ["/content/sample_data/fileToAnalyze_OnlyAns.json"]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [3]:
assistant = client.beta.assistants.update(
assistant_id=assistant.id,
tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [4]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
file=open("/content/sample_data/fileToAnalyze_OnlyAns.json", "rb"), purpose="assistants")

# Create a thread and attach the file to the message
thread = client.beta.threads.create(
messages=[
  {
    "role": "user",
    "content": "Analyze all comments and fetch categories, sub-category and keywords for each open answer",
    # Attach the new file to the message.
    "attachments": [
      { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
    ],
  }
]
)

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_AeHhoeeTw09ixhnjmhWe3wdM'])


In [6]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI

#client = OpenAI()

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
      print(f"\nassistant > ", end="", flush=True)

  @override
  def on_tool_call_created(self, tool_call):
      print(f"\nassistant > {tool_call.type}\n", flush=True)

  @override
  def on_message_done(self, message) -> None:
      # print a citation to the file searched
      message_content = message.content[0].text
      annotations = message_content.annotations
      citations = []
      for index, annotation in enumerate(annotations):
          message_content.value = message_content.value.replace(
              annotation.text, f"[{index}]"
          )
          if file_citation := getattr(annotation, "file_citation", None):
              cited_file = client.files.retrieve(file_citation.file_id)
              citations.append(f"[{index}] {cited_file.filename}")

      print(message_content.value)
      print("\n".join(citations))


# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > file_search


assistant > Here is the analysis of the comments with categories, sub-categories, and keywords for each open answer:

1. **Category**: Pricing  
   - **Sub-category**: High Prices  
   - **Keywords**: high prices, quality, value, cost[0]

2. **Category**: Transparency  
   - **Sub-category**: Hidden Fees  
   - **Keywords**: pricing, hidden fees, transparency, deceptive[0]

3. **Category**: Delivery  
   - **Sub-category**: Delays  
   - **Keywords**: delivery, express shipping, late, updates[0]

4. **Category**: Packaging  
   - **Sub-category**: Damaged  
   - **Keywords**: packaging, damaged, securely, quality[0]

5. **Category**: Product Quality  
   - **Sub-category**: Low Quality  
   - **Keywords**: low quality, cheap, brand[0]

6. **Category**: Usability  
   - **Sub-category**: Complicated Products  
   - **Keywords**: complicated, instructions, user-friendly[0]

7. **Category**: Customer Service  
   - **Sub-category**: Care and Empathy  
   - **Key